In [1]:
import os
import tensorflow as tf
import sklearn
import keras
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose, UpSampling2D, concatenate, Input, Cropping2D
from keras.models import Model
from keras.optimizers import Adam
import math
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from skimage.io import imread
from keras.models import load_model
from keras.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K

Using TensorFlow backend.


## 1. Read Images

In [2]:
TRAIN_IMG_PATH = 'banco_dados_SB/train/imagens/'
TRAIN_LBL_PATH = 'banco_dados_SB/train/classes/'

TEST_IMG_PATH = 'banco_dados_SB/test/imagens/'
TEST_LBL_PATH = 'banco_dados_SB/test/classes/'

In [3]:
X_train = [imread(fname=TRAIN_IMG_PATH + img, as_grey=True) for img in os.listdir(TRAIN_IMG_PATH)]
y_train = [imread(fname=TRAIN_LBL_PATH + lbl)[:, :, 0] for lbl in os.listdir(TRAIN_LBL_PATH)]
X_test = [imread(fname=TEST_IMG_PATH + img, as_grey=True) for img in os.listdir(TEST_IMG_PATH)]
y_test = [imread(fname=TEST_LBL_PATH + lbl)[:, :, 0] for lbl in os.listdir(TEST_LBL_PATH)]

In [4]:
from pandas import Series, DataFrame
import pandas as pd

## 2. Criando o objeto Series e exportando para excel

In [5]:
obj = Series(os.listdir(TRAIN_IMG_PATH))

In [6]:
obj.to_excel('teste.xls')

In [7]:
obj2 = Series(os.listdir(TEST_IMG_PATH))
obj2.to_excel('teste2.xls')

## 3. importando excel e customizando

In [80]:
teste = pd.read_excel('teste3.xls')

In [87]:
teste.head()

,images,type
0,1.png,HFNI
1,10.png,HFNI
2,11.png,HFNI
3,145.png,N
4,146.png,N


In [88]:
teste['type'].value_counts()

HFNI    19
HFI     10
N        9
HYP      6
Name: type, dtype: int64

In [89]:
from sklearn.model_selection import StratifiedShuffleSplit

In [90]:
split = StratifiedShuffleSplit (n_splits=1, test_size=0.2, random_state = 42)
for train_index,validation_index in split.split(teste,teste['type']):
    strat_train_set = teste.iloc[train_index]
    strat_valid_set = teste.iloc[validation_index]

In [91]:
print(teste['type'].value_counts() /len(teste))
print('tamanho do set imagens:',len(teste))

HFNI    0.431818
HFI     0.227273
N       0.204545
HYP     0.136364
Name: type, dtype: float64
tamanho do set imagens: 44


In [92]:
print(strat_train_set['type'].value_counts() /len(strat_train_set))
print('tamanho de treino:',len(strat_train_set))

HFNI    0.428571
HFI     0.228571
N       0.200000
HYP     0.142857
Name: type, dtype: float64
tamanho de treino: 35


In [93]:
print(strat_valid_set['type'].value_counts() /len(strat_valid_set))
print('tamanho de validação:',len(strat_valid_set))

HFNI    0.444444
HFI     0.222222
N       0.222222
HYP     0.111111
Name: type, dtype: float64
tamanho de validação: 9


In [94]:
strat_train_set.head()

,images,type
1,10.png,HFNI
29,233.png,HFNI
16,205.png,HFI
31,235.png,HFNI
39,406.png,HYP


In [95]:
for set_ in (strat_train_set,strat_valid_set):
    set_.drop('type',axis=1,inplace=True)

C:\Users\Tales\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
strat_train_set.head()

,images
1,10.png
29,233.png
16,205.png
31,235.png
39,406.png


In [97]:
list_teste = list(strat_train_set['images'])

In [98]:
list_teste

['10.png',
 '233.png',
 '205.png',
 '235.png',
 '406.png',
 '2.png',
 '1.png',
 '199.png',
 '11.png',
 '229.png',
 '405.png',
 '210.png',
 '301.png',
 '206.png',
 '409.png',
 '147.png',
 '407.png',
 '202.png',
 '212.png',
 '296.png',
 '232.png',
 '231.png',
 '197.png',
 '146.png',
 '149.png',
 '150.png',
 '215.png',
 '411.png',
 '196.png',
 '211.png',
 '297.png',
 '214.png',
 '234.png',
 '208.png',
 '203.png']